# Pulling Data from Google Earth Engine
Here, I try to:
- visualize the image I am trying to pull using Folium; and 
- export the image in the GeoTIFF format.

In [72]:
# Import packages
import ee
import pandas as pd
import numpy as np
import time
from geetools import batch
from tqdm import tqdm
from functools import partial
import folium
import rasterio as rio
from pyproj import Proj, transform

In [73]:
# Personal credentials
service_account = "sentinel2@uhi-causal-model.iam.gserviceaccount.com"
json_key = "uhi-causal-model-a7e891b460a5.json"

In [74]:
# Initialize the library
ee.Initialize(ee.ServiceAccountCredentials(service_account, json_key), opt_url='https://earthengine-highvolume.googleapis.com')

In [75]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

## Recover the bounds from TIF file

The bounds will be extracted from the traversal data `Evening_Area-wide_Temperature_Boston.tif` to maintain across all data.

In [76]:
# Load the bounds from the Tiff file
tif_path = "data\Boston\Evening_Area-wide_Temperature_Boston.tif"
with rio.open(tif_path) as src:
    bounds = src.bounds
    crs = src.crs
print(f'Bounds in {src.crs} projection: {bounds}')

Bounds in EPSG:32619 projection: BoundingBox(left=318730.0, bottom=4676540.0, right=338310.0, top=4698980.0)


<>:2: SyntaxWarning: invalid escape sequence '\E'
<>:2: SyntaxWarning: invalid escape sequence '\E'
C:\Users\esl26\AppData\Local\Temp\ipykernel_21496\511444451.py:2: SyntaxWarning: invalid escape sequence '\E'
  tif_path = "data\Evening_Area-wide_Temperature_Boston.tif"


In [77]:
# Convert coordinate to EPSG: 3857 projection
source_crs = Proj(init='EPSG:32619')
target_crs = Proj(init='EPSG:3857')

print(f'Bounds with 32619: ', [bounds.left, bounds.bottom, bounds.right, bounds.top])

left, bottom  = transform(source_crs, target_crs, bounds.left, bounds.bottom)
right, top = transform(source_crs, target_crs, bounds.right, bounds.top)
new_bounds = [left, bottom, right, top]

print(f'Bounds with 3857: ', new_bounds)

Bounds with 32619:  [318730.0, 4676540.0, 338310.0, 4698980.0]
Bounds with 3857:  [-7925537.469545445, 5193993.849869293, -7899843.3450095495, 5225051.230627105]


c:\Users\esl26\AppData\Local\anaconda3\envs\research\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\esl26\AppData\Local\anaconda3\envs\research\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\esl26\AppData\Local\Temp\ipykernel_21496\24135500.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.ht

In [78]:
# Add a buffer to the bounds (in meters) to make sure that we have sufficient space around the traversal path.
# I recommend a buffer of 1000 meters, but to be conservative, you could make it 2000 meters, too. 
new_bounds = [new_bounds[0]-1000, new_bounds[1]-1000, new_bounds[2]+1000, new_bounds[3]+1000]
print('after', new_bounds)

after [-7926537.469545445, 5192993.849869293, -7898843.3450095495, 5226051.230627105]


In [96]:
# Import dataset
sentinel = ee.ImageCollection("COPERNICUS/S2_SR")

# Define parameters: temporal and spatial
startDate = '2019-07-29'
endDate = '2019-07-30'
lat,lon = 42.325347,-71.120802
crs = 'EPSG:3857'

# Define geometry
xMin, xMax, yMin, yMax = new_bounds[0], new_bounds[2], new_bounds[1], new_bounds[3]
geometry = ee.Geometry.Rectangle([[xMin, yMin],[xMax, yMax]], crs, False, True)

## Folium Visualizations

### True color bands

In [97]:
# Visualizer
RGB_VIZ = {'min':0, 'max':255, 'bands':['TCI_R', 'TCI_G', 'TCI_B']}

# Display true color image
filtered = sentinel.filterDate(startDate, endDate).filterBounds(geometry).select(['TCI_R', 'TCI_G', 'TCI_B'])
image = filtered.median().clip(geometry)
map = folium.Map(location=[lat, lon], zoom_start=11)
map.add_ee_layer(image, RGB_VIZ, 'rgb')
display(map)

### NDVI

In [108]:
filtered = sentinel.filterDate(startDate, endDate).filterBounds(geometry).select(['B8', 'B4'])
# filtered = filtered.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',25))
image = filtered.median().clip(geometry)
ndvi = image.normalizedDifference(['B8', 'B4'])
map = folium.Map(location=[lat, lon], zoom_start=11)
ndvi_params = {'min': -1, 'max': 1, 'palette': ['#640000','#ff0000','#ffff00','#00c800','#006400']}
map.add_ee_layer(ndvi, ndvi_params, 'NDVI')
display(map)

### SCL (raw band from Sentinel-2)

In [102]:
filtered = sentinel.filterDate(startDate, endDate).filterBounds(geometry).select(['SCL'])
image = filtered.median().clip(geometry)
map = folium.Map(location=[lat, lon], zoom_start=11)
scl_params = {'min': 1, 'max': 11, 'bands': ['SCL'], 'palette':['#ff0004','#868686','#774b0a','#10d22c','#ffff52','#0000ff','#818181','#c0c0c0','#f1f1f1','#bac5eb','#52fff9']}
map.add_ee_layer(image, scl_params, 'SCL')
display(map)

## Exporting images

In [103]:
task = ee.batch.Export.image.toCloudStorage(
    image=ndvi,
    region=geometry,
    description="boston_ndvi",
    crs = crs,
    maxPixels=1e13,
    scale=10,
    bucket='earth-engine_boston',
    fileFormat='GeoTIFF'
)

task.start()

In [109]:
# While the task is running, you can check on its status...it may take a few minutes
# to fully load the image.
task.status()

{'state': 'COMPLETED',
 'description': 'boston_ndvi',
 'creation_timestamp_ms': 1708980682856,
 'update_timestamp_ms': 1708980842437,
 'start_timestamp_ms': 1708980688795,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://console.developers.google.com/storage/browser/earth-engine_boston/'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 39.49863052368164,
 'id': 'AJTL27BAA5SN5YAQSVI7TQHL',
 'name': 'projects/earthengine-legacy/operations/AJTL27BAA5SN5YAQSVI7TQHL'}